In [1]:
import datetime
import numpy as np
import pandas as pd
import pyarrow.dataset as ds
import pyarrow as pa
from datetime import datetime

In [2]:
def fix_schema(table):
    """
    This
    :param table:
    :return:
    """
    schema = table.schema
    field_names = ['dropOff_datetime', 'pickup_datetime', 'tpep_dropoff_datetime', 'tpep_pickup_datetime']
    field_index = [schema.get_field_index(name) for name in field_names]
    # set new fields
    for i in range(len(field_names)):
        if not field_index[i] == -1:
            schema = schema.set(field_index[i], pa.field(field_names[i], pa.int64()))
    table = table.cast(target_schema=schema)
    return table

In [3]:

file = "C:\\Users\\Tim Garbe\\IdeaProjects\\DWH_NYC_Taxi\\data\\taxi\\fhv_01_2022.parquet"

schema = pa.schema([
    ('dispatching_base_num', pa.string()),
    ('dropOff_datetime', pa.int64()),
    ('pickup_datetime', pa.int64()),
    ('PUlocationID', pa.int64()),
    ('DOlocationID', pa.int64()),
    ('PULocationID', pa.int64()),
    ('DOLocationID', pa.int64()),
    ('SR_Flag', pa.float64()),
    ('painis', pa.float64()),
    ('Affiliated_base_number', pa.string()),
])
table = ds.dataset(file).to_table()
df_new = fix_schema(table).to_pandas()

In [4]:
df_new


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,1640997060000000,1640999100000000,NaN,NaN,None,B00009
1,B00009,1640997420000000,1640999100000000,NaN,NaN,None,B00009
2,B00037,1640998597000000,1640999171000000,NaN,85.0,None,B00037
3,B00037,1640996394000000,1640997047000000,NaN,85.0,None,B00037
4,B00037,1640997709000000,1640998336000000,NaN,188.0,None,B00037
...,...,...,...,...,...,...,...
1143686,B03380,1643671361000000,1643671599000000,234.0,107.0,None,B03380
1143687,B03380,1643672562000000,1643673178000000,114.0,148.0,None,B03380
1143688,B03380,1643670433000000,1643670820000000,90.0,113.0,None,B03380
1143689,B03380,1643670974000000,1643671863000000,113.0,140.0,None,B03380


In [5]:
df = df_new.copy()

df['date'] = df['pickup_datetime'].apply(lambda x: pd.to_datetime(x, unit='us').strftime('%Y-%m-%d'))

In [6]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,date
0,B00009,1640997060000000,1640999100000000,NaN,NaN,None,B00009,2022-01-01
1,B00009,1640997420000000,1640999100000000,NaN,NaN,None,B00009,2022-01-01
2,B00037,1640998597000000,1640999171000000,NaN,85.0,None,B00037,2022-01-01
3,B00037,1640996394000000,1640997047000000,NaN,85.0,None,B00037,2022-01-01
4,B00037,1640997709000000,1640998336000000,NaN,188.0,None,B00037,2022-01-01
...,...,...,...,...,...,...,...,...
1143686,B03380,1643671361000000,1643671599000000,234.0,107.0,None,B03380,2022-01-31
1143687,B03380,1643672562000000,1643673178000000,114.0,148.0,None,B03380,2022-01-31
1143688,B03380,1643670433000000,1643670820000000,90.0,113.0,None,B03380,2022-01-31
1143689,B03380,1643670974000000,1643671863000000,113.0,140.0,None,B03380,2022-01-31


In [7]:
timetamp = int(1640998480000000)
print(timetamp)

1640998480000000


In [8]:
pd.to_datetime(timetamp, unit='us')


Timestamp('2022-01-01 00:54:40')

In [9]:
df.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number',
       'date'],
      dtype='object')

In [10]:
meta_data = {
    'PUlocationID' : 'value_count',
    'DOlocationID' : 'value_count',
}

In [11]:
def get_value_counts_as_columns(df_group, feature):
    ret_df = pd.DataFrame()
    for group in df_group.groups:
        sub_df = df_group.get_group(group) # .groupby('date')#
        sub_df = sub_df[sub_df[feature].notna()]
        sub_df = sub_df[feature].value_counts().reset_index()
        sub_df['index'] = sub_df['index'].apply(lambda x: f'{feature}_{int(x)}')
        sub_df = sub_df.swapaxes(0, 1)
        sub_df.columns = list(sub_df.iloc[0])
        sub_df = sub_df.drop('index')
        sub_df.index = [group]
        ret_df = pd.concat([ret_df, sub_df], axis=0)
    return ret_df

df_group = df.groupby('date')
df_daily = pd.DataFrame()
for feature in meta_data:
    if feature in list(df.columns):
        combine_type = meta_data[feature]
        if combine_type == 'value_count':
            df_new = get_value_counts_as_columns(df_group, feature)
        else:
            df_new = df_group[feature]
            df_new = getattr(df_new, combine_type)()
            df_new.columns = [feature]
        df_daily = pd.concat([df_daily, df_new], axis=1)

In [12]:
df_daily.head()

,PUlocationID_221,PUlocationID_56,PUlocationID_206,PUlocationID_132,PUlocationID_129,PUlocationID_92,PUlocationID_82,PUlocationID_173,PUlocationID_115,PUlocationID_7,...,DOlocationID_27,DOlocationID_5,DOlocationID_207,DOlocationID_12,DOlocationID_8,DOlocationID_99,DOlocationID_30,DOlocationID_2,DOlocationID_105,DOlocationID_199
2022-01-01,150,126,107,101,97,96,81,79,77,56,...,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-02,142,119,129,104,154,137,118,81,63,69,...,NaN,3,5,1,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03,204,163,195,84,190,211,129,106,89,126,...,NaN,9,3,1,3,NaN,NaN,NaN,NaN,NaN
2022-01-04,222,148,205,60,213,190,121,89,113,108,...,1,14,NaN,2,2,2,NaN,NaN,NaN,NaN
2022-01-05,196,148,203,74,202,214,135,98,91,118,...,2,8,4,3,NaN,1,1,NaN,NaN,NaN


In [22]:
df_daily

,PUlocationID_221,PUlocationID_56,PUlocationID_206,PUlocationID_132,PUlocationID_129,PUlocationID_92,PUlocationID_82,PUlocationID_173,PUlocationID_115,PUlocationID_7,...,DOlocationID_27,DOlocationID_5,DOlocationID_207,DOlocationID_12,DOlocationID_8,DOlocationID_99,DOlocationID_30,DOlocationID_2,DOlocationID_105,DOlocationID_199
2022-01-01,150,126,107,101,97,96,81,79,77,56,...,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-02,142,119,129,104,154,137,118,81,63,69,...,NaN,3,5,1,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03,204,163,195,84,190,211,129,106,89,126,...,NaN,9,3,1,3,NaN,NaN,NaN,NaN,NaN
2022-01-04,222,148,205,60,213,190,121,89,113,108,...,1,14,NaN,2,2,2,NaN,NaN,NaN,NaN
2022-01-05,196,148,203,74,202,214,135,98,91,118,...,2,8,4,3,NaN,1,1,NaN,NaN,NaN
2022-01-06,190,137,211,65,216,233,130,100,85,143,...,3,8,3,1,3,NaN,NaN,NaN,NaN,NaN
2022-01-07,165,109,111,84,157,168,102,91,66,91,...,1,7,3,NaN,1,1,2,NaN,NaN,NaN
2022-01-08,168,145,145,80,194,172,124,125,73,109,...,1,5,3,1,1,1,NaN,NaN,NaN,NaN
2022-01-09,121,148,119,67,140,113,100,119,72,81,...,1,5,2,NaN,4,3,NaN,NaN,NaN,NaN
2022-01-10,193,159,190,64,223,231,139,119,99,145,...,1,11,1,1,1,4,2,NaN,NaN,NaN


In [13]:
df7 = df.copy()

In [14]:
df7 = df7.groupby(['date', 'PUlocationID'])

In [15]:
type(df7)

pandas.core.groupby.generic.DataFrameGroupBy

In [16]:
df_new = df7['PUlocationID']

In [17]:
combine_type = 'count'
df_new = getattr(df_new, combine_type)()

In [18]:
df_new.head()

date        PUlocationID
2022-01-01  1.0             16
            3.0              1
            4.0             12
            5.0              3
            6.0             11
Name: PUlocationID, dtype: int64

In [19]:
feature = 'PUlocationID'
date = '2022-01-01'
df3 = df[feature].value_counts().reset_index()
df3['index'] = df3['index'].apply(lambda x: f'{feature}_{int(x)}')
df3

,index,PUlocationID
0,PUlocationID_129,6151
1,PUlocationID_92,5569
2,PUlocationID_221,5421
3,PUlocationID_206,4992
4,PUlocationID_56,4312
...,...,...
254,PUlocationID_99,14
255,PUlocationID_30,8
256,PUlocationID_12,7
257,PUlocationID_110,2


In [20]:
df3 = df3.swapaxes(0, 1)
df3


,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,256,257,258
index,PUlocationID_129,PUlocationID_92,PUlocationID_221,PUlocationID_206,PUlocationID_56,PUlocationID_82,PUlocationID_165,PUlocationID_123,PUlocationID_210,PUlocationID_7,...,PUlocationID_34,PUlocationID_253,PUlocationID_120,PUlocationID_184,PUlocationID_8,PUlocationID_99,PUlocationID_30,PUlocationID_12,PUlocationID_110,PUlocationID_111
PUlocationID,6151,5569,5421,4992,4312,3941,3884,3758,3678,3650,...,20,20,18,15,15,14,8,7,2,1


In [21]:
df3.columns = list(df3.iloc[0])
df3 = df3.drop('index')
df3.index = [date]
df3

,PUlocationID_129,PUlocationID_92,PUlocationID_221,PUlocationID_206,PUlocationID_56,PUlocationID_82,PUlocationID_165,PUlocationID_123,PUlocationID_210,PUlocationID_7,...,PUlocationID_34,PUlocationID_253,PUlocationID_120,PUlocationID_184,PUlocationID_8,PUlocationID_99,PUlocationID_30,PUlocationID_12,PUlocationID_110,PUlocationID_111
2022-01-01,6151,5569,5421,4992,4312,3941,3884,3758,3678,3650,...,20,20,18,15,15,14,8,7,2,1
